In [1]:
from calUtils import *
from scipy.optimize import least_squares, minimize

In [2]:
file_num = 18
folder = 'iProbeCalibration/geoCalRobot3'
def err_iprobe(iprobe_prm_input, solidCube_prm_input, mode):
    return getError_robot(Prm(iprobe_prm_input[0], 
                        iprobe_prm_input[1], 
                        iprobe_prm_input[2]), 
                    solidCube(solidCube_prm_input[0], 
                              solidCube_prm_input[1], 
                              solidCube_prm_input[2], 
                              solidCube_prm_input[3], 
                              solidCube_prm_input[4], 
                              solidCube_prm_input[5]), 
                    mode, file_num, folder)
    
def err_solidcube(solidCube_prm_input, iprobe_prm_input, mode):
    return getError_robot(Prm(iprobe_prm_input[0], 
                        iprobe_prm_input[1], 
                        iprobe_prm_input[2]), 
                    solidCube(solidCube_prm_input[0], 
                              solidCube_prm_input[1], 
                              solidCube_prm_input[2], 
                              solidCube_prm_input[3], 
                              solidCube_prm_input[4], 
                              solidCube_prm_input[5]), 
                    mode, file_num, folder)

In [3]:
def iterative_optimization(iprobe_prm_input, solidCube_prm_input, 
                           tol=1e-6, max_iter=1000):
    prev_error = np.inf
    for iteration in range(max_iter):
        result_iprobe = minimize(fun=err_iprobe, 
                                 x0=iprobe_prm_input, 
                                 args=(solidCube_prm_input, 'scalar'), 
                                 method='L-BFGS-B', 
                                 bounds=[(53.5, 73.5), 
                                         (67.0, 87.0), 
                                         (-87.0, -67.0)], 
                                 tol=tol,
                                 options={'maxiter': max_iter, 'disp': False})
        iprobe_prm_input = result_iprobe.x
        
        result_solidCube = least_squares(fun=err_solidcube, 
                                         x0=solidCube_prm_input, 
                                         args=(iprobe_prm_input, 'vector'), 
                                         ftol=tol, 
                                         # tr_options={'verbosity': 1, 'max_iter': 2000}, 
                                         verbose=1, 
                                         max_nfev=max_iter)
        solidCube_prm_input = result_solidCube.x
        
        current_error = err_iprobe(iprobe_prm_input, solidCube_prm_input, 'scalar')
        print(f"Iteration {iteration+1}, Error: {current_error:.8f}")
        if abs(prev_error - current_error) < tol:
            print("Converged.")
            break
        elif current_error >= prev_error:
            print("Converged.")
            return prev_iprobe_prm, prev_solidCube_prm, prev_error
        prev_error = current_error
        prev_iprobe_prm = iprobe_prm_input
        prev_solidCube_prm = solidCube_prm_input
    
    return iprobe_prm_input, solidCube_prm_input, current_error
    

In [4]:
iprobe_prm_input = [63.5, 77.0, -77.0]
solidCube_prm_input = [-2.2266e-03, 6.3928e-02, -5.1277e-05, 
                       5.4682e+00, 7.0481e-04, 4.8222e-04]
# iprobe_prm_input = [66.20844612, 74.07940723, -77.77190349]
# solidCube_prm_input = [5.75713355e-03, 1.65454347e-01, -2.51927934e-03, 
#                        5.47618023e+00, 6.33467621e-02, -4.00017302e-04]
# iprobe_prm_input = [65.39392092546034, 74.27754179794745, -78.90018349612558]
# solidCube_prm_input = [0.026621052826668653, 0.28142488671966015, -0.003442528937855944, 
#                        5.497038874449061, 0.004936267489441483, -0.0005406968424569496]

print(f"Before calibration: {err_iprobe(iprobe_prm_input, solidCube_prm_input, 'scalar')}")
final_iprobe_prm, final_solidCube_prm, final_error = iterative_optimization(iprobe_prm_input, solidCube_prm_input)

print("\nFinal optimized parameters:")
print(f"iProbe Parameters: [{final_iprobe_prm[0]}, {final_iprobe_prm[1]}, {final_iprobe_prm[2]}]")
print(f"SolidCube Parameters: [{final_solidCube_prm[0]}, {final_solidCube_prm[1]}, {final_solidCube_prm[2]}, {final_solidCube_prm[3]}, {final_solidCube_prm[4]}, {final_solidCube_prm[5]}]")
print("Final Error:", final_error)

Before calibration: 4.112481225581502
The maximum number of function evaluations is exceeded.
Function evaluations 1000, initial cost 2.5693e+00, final cost 1.8076e+00, first-order optimality 1.27e+00.
Iteration 1, Error: 1.90136010
The maximum number of function evaluations is exceeded.
Function evaluations 1000, initial cost 1.7017e+00, final cost 1.6620e+00, first-order optimality 1.35e+00.
Iteration 2, Error: 1.82318968
The maximum number of function evaluations is exceeded.
Function evaluations 1000, initial cost 1.6538e+00, final cost 1.6399e+00, first-order optimality 1.38e+00.
Iteration 3, Error: 1.81103937
The maximum number of function evaluations is exceeded.
Function evaluations 1000, initial cost 1.6392e+00, final cost 1.5339e+00, first-order optimality 3.42e+00.
Iteration 4, Error: 1.75153208
The maximum number of function evaluations is exceeded.
Function evaluations 1000, initial cost 1.5190e+00, final cost 1.4887e+00, first-order optimality 3.73e+00.
Iteration 5, Error

In [19]:
# test
iprobe_prm_input = [63.5, 77, -77]
solidCube_prm_input = [0, 0, 0, 0, 0, 0]
print(f"Before calibration: {err_iprobe(iprobe_prm_input, solidCube_prm_input, 'scalar')}")

iprobe_prm_input = [64.5011708628196, 75.26929508546563, -77.89281550217865]
solidCube_prm_input = [2.7855151612228064, -0.07886438612558434, 0.0028889278738881566, 
                       8.261248615052335, -0.07499427477443624, 0.0017110750522519528]
print(f"After calibration: {err_iprobe(iprobe_prm_input, solidCube_prm_input, 'scalar')}")


Before calibration: 6.815403738658085
After calibration: 1.6111992777845625
